**Table of contents**<a id='toc0_'></a>    
- [Robot Arm](#toc1_)    
- [Starting pybullet](#toc2_)    
- [Initial Setup for pybullet](#toc3_)    
- [Generating the Robot Arm](#toc4_)    
- [Defining Functions](#toc5_)    
  - [Defining Homogeneous Transformation Matrix](#toc5_1_)    
  - [Defining the Jacobian Matrix](#toc5_2_)    
  - [Defining the Pseudo-Inverse Matrix](#toc5_3_)    
- [Running the Simulation](#toc6_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# <a id='toc1_'></a>[Robot Arm](#toc0_)

In this notebook, we will generate a 2-axis robot arm and explain how to solve inverse kinematics using numerical methods with the Jacobian matrix.

![](../images/RobotArm/robot_arm_jacobian_inverse_kinemarics/inverse_kinematics_jacobian_overview_en.png)

(For a manual summarizing the functions available in pybullet, please refer to [this link](https://github.com/bulletphysics/bullet3/blob/master/docs/pybullet_quickstartguide.pdf).)

# <a id='toc2_'></a>[Starting pybullet](#toc0_)

In [1]:
import pybullet
import pybullet_data
physics_client = pybullet.connect(pybullet.GUI) 

startThreads creating 1 threads.
starting thread 0


pybullet build time: Nov 28 2023 23:45:17


started thread 0 
argc=2
argv[0] = --unused
argv[1] = --start_demo_name=Physics Server
ExampleBrowserThreadFunc started
X11 functions dynamically loaded using dlopen/dlsym OK!
X11 functions dynamically loaded using dlopen/dlsym OK!
Creating context
Created GL 3.3 context
Direct GLX rendering context obtained
Making context current
GL_VENDOR=Mesa
GL_RENDERER=llvmpipe (LLVM 15.0.7, 256 bits)
GL_VERSION=4.5 (Core Profile) Mesa 23.2.1-1ubuntu3.1~22.04.2
GL_SHADING_LANGUAGE_VERSION=4.50
pthread_getconcurrency()=0
Version = 4.5 (Core Profile) Mesa 23.2.1-1ubuntu3.1~22.04.2
Vendor = Mesa
Renderer = llvmpipe (LLVM 15.0.7, 256 bits)
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started


# <a id='toc3_'></a>[Initial Setup for pybullet](#toc0_)

In [2]:
pybullet.resetSimulation() # Reset the simulation space
pybullet.setAdditionalSearchPath(pybullet_data.getDataPath()) # Add path to necessary data for pybullet
pybullet.setGravity(0.0, 0.0, -9.8) # Set gravity as on Earth
time_step = 1./240.
pybullet.setTimeStep(time_step) # Set the elapsed time per step

# Load the floor
plane_id = pybullet.loadURDF("plane.urdf")

# Set the camera position for GUI mode
camera_distance = 2.0
camera_yaw = 0.0 # deg
camera_pitch = -20 # deg
camera_target_position = [0.0, 0.0, 0.0]
pybullet.resetDebugVisualizerCamera(camera_distance, camera_yaw, camera_pitch, camera_target_position)

ven = Mesa
ven = Mesa


# <a id='toc4_'></a>[Generating the Robot Arm](#toc0_)
This time, we will generate a 2-axis robot arm `simple_2d_arm.urdf`.  
The robot is configured as shown in the diagram below (sensors are explained in `robot_arm_sensor.ipynb`).  

![](../images/RobotArm/2d_robot_arm.png)

In [3]:
# Load the robot
arm_start_pos = [0, 0, 0.1]  # Set initial position (x, y, z)
arm_start_orientation = pybullet.getQuaternionFromEuler([0, 0, 0])  # Set initial orientation (roll, pitch, yaw)
arm_id = pybullet.loadURDF("../urdf/simple2d_arm.urdf", arm_start_pos, arm_start_orientation, useFixedBase=True)  # Fix the root link with useFixedBase=True to prevent the robot from falling

# Set the camera position for GUI mode
camera_distance = 1.5
camera_yaw = 180.0  # deg
camera_pitch = -10  # deg
camera_target_position = [0.0, 0.0, 1.0]
pybullet.resetDebugVisualizerCamera(camera_distance, camera_yaw, camera_pitch, camera_target_position)

b3Printf: b3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:

b3Printf: No inertial data for link, using mass=1, localinertiadiagonal = 1,1,1, identity local inertial frame
b3Printf: b3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:

b3Printf: target_position_vertual_link


# <a id='toc5_'></a>[Defining Functions](#toc0_)

## <a id='toc5_1_'></a>[Defining Homogeneous Transformation Matrix](#toc0_)
We will define functions to create homogeneous transformation matrices.

(For more details, please refer to [robot_arm_homogeneous_matrix_forward_kinematics](https://github.com/akinami3/PybulletRobotics/blob/main/RobotArm/robot_arm_homogeneous_matrix_forward_kinematics_en.ipynb).)

In [4]:
import numpy as np

def Hz(theta):
    """
    Calculate the rotation matrix in a 2D plane (rotation around the z-axis in a 2D plane)

    Parameters
    ----------
    theta : float
        Rotation angle [rad]

    Returns
    -------
    numpy.ndarray
        Rotation matrix
    """
    return np.array([[np.cos(theta), -np.sin(theta), 0], 
                     [np.sin(theta), np.cos(theta), 0], 
                     [0, 0, 1]])

def Hp(x, y):
    """
    Calculate the translation matrix in a 2D plane

    Parameters
    ----------
    x : float
        Translation amount in the x direction
    y : float
        Translation amount in the y direction

    Returns
    -------
    numpy.ndarray
        Translation matrix
    """
    return np.array([[1, 0, x], 
                     [0, 1, y], 
                     [0, 0, 1]])

## <a id='toc5_2_'></a>[Defining the Jacobian Matrix](#toc0_)
In [robot_arm_analytical_inverse_kinematics.ipynb](../RobotArm/robot_arm_analytical_inverse_kinematics_en.ipynb), we solved inverse kinematics analytically.

Analytical solutions for inverse kinematics can provide instant solutions, but it can be challenging to solve analytically when the robot arm's configuration is complex.

<br>

On the other hand, numerical methods for inverse kinematics solve the problem by gradually moving each joint angle $\vec{Q}$ to approach the "target end-effector position $\vec{P_{\mathrm{goal}}}$", making it possible to find solutions even for complex robot arm configurations.

![](../images/RobotArm/robot_arm_jacobian_inverse_kinemarics/inverse_kinematics_jacobian_overview_en.png)

<br>

In this process, it is necessary to relate the "small change in end-effector position $\Delta \vec{P}$" to the "small change in joint angles $\Delta \vec{Q}$", and this is where the "Jacobian matrix $J$" comes into play.
$$
\Delta \vec{P} = J \Delta \vec{Q}
$$
![](../images/RobotArm/robot_arm_jacobian_inverse_kinemarics/delta_q_delta_p_jacpbian_en.png)

<br>

The Jacobian matrix is defined as the partial derivatives of the forward kinematics equations with respect to each joint angle.
$$
J = \begin{bmatrix}
\frac{\partial x}{\partial \theta_1} & \frac{\partial x}{\partial \theta_2} & \cdots & \frac{\partial x}{\partial \theta_n} \\
\frac{\partial y}{\partial \theta_1} & \frac{\partial y}{\partial \theta_2} & \cdots & \frac{\partial y}{\partial \theta_n} \\
\frac{\partial z}{\partial \theta_1} & \frac{\partial z}{\partial \theta_2} & \cdots & \frac{\partial z}{\partial \theta_n} \\
\end{bmatrix}
$$

<br>

For the 2-axis robot arm in this case, the forward kinematics equations are
$$
x = l_1 \cos(\theta_1) + l_2 \cos(\theta_1 + \theta_2) \\
y = l_1 \sin(\theta_1) + l_2 \sin(\theta_1 + \theta_2) \\
$$
Therefore, the Jacobian matrix is as follows:
$$
J = \begin{bmatrix}
\frac{\partial x}{\partial \theta_1} & \frac{\partial x}{\partial \theta_2} \\
\frac{\partial y}{\partial \theta_1} & \frac{\partial y}{\partial \theta_2} \\
\end{bmatrix}
= \begin{bmatrix}
-l_1 \sin(\theta_1) - l_2 \sin(\theta_1 + \theta_2) & -l_2 \sin(\theta_1 + \theta_2) \\
l_1 \cos(\theta_1) + l_2 \cos(\theta_1 + \theta_2) & l_2 \cos(\theta_1 + \theta_2) \\
\end{bmatrix}
$$

---

- $l_1$: Length of link 1
- $l_2$: Length of link 2
- $\theta_1$: Angle of joint 1
- $\theta_2$: Angle of joint 2

---

<br>

Now that we have $J$, to solve
$$\Delta \vec{P} = J \Delta \vec{Q} (revisited)$$
we need to find either $\Delta \vec{P}$ or $\Delta \vec{Q}$.  
$\Delta \vec{P}$ can be easily found as the difference between the "target end-effector position $\vec{P_{\mathrm{goal}}}$" and the "current end-effector position $\vec{P_{\mathrm{current}}}$", as shown below.
($P_{\mathrm{param}}$ is a parameter that determines how much to change the end-effector position.)
$$
\Delta \vec{P} = P_{\mathrm{param}}(\vec{P_{\mathrm{goal}}} - \vec{P_{\mathrm{current}}})
$$
![](../images/RobotArm/robot_arm_jacobian_inverse_kinemarics/delta_p_en.png)
* Here, $P_{\mathrm{param}}$ is treated as a scalar value, but it can also be set as $P_{\mathrm{param}} = \begin{bmatrix} P_{\mathrm{param_x}} & 0 \\ 0 & P_{\mathrm{param_y}} \end{bmatrix}$ with different values for the x and y components.

<br>

With the "small change in end-effector position $\Delta \vec{P}$" and the "Jacobian matrix $J$" determined, we can find the "small change in joint angles $\Delta \vec{Q}$" through the following transformation. (Here, $J^+$ represents the pseudo-inverse of the Jacobian matrix $J$.)
$$
\Delta \vec{P} = J \Delta \vec{Q} \\
\Delta \vec{Q} = J^+ \Delta \vec{P}
$$

<br>

Using the above equations, we can gradually move the joint angles $\vec{Q}$ until the "current end-effector position $\vec{P_{\mathrm{current}}}$" converges to the "target end-effector position $\vec{P_{\mathrm{goal}}}$".


In [5]:
import math
def make_jacobian_matrix(link1_length, theta1, link2_length, theta2):
    """
    Calculate the Jacobian matrix of a 2-link arm in a 2D plane
    J = [[∂x/∂θ1, ∂x/∂θ2],
         [∂y/∂θ1, ∂y/∂θ2]]

    Parameters
    ----------
    link1_length : float
        Length of link 1
    link2_length : float
        Length of link 2
    theta1 : float
        Rotation angle of link 1 (rad)
    theta2 : float
        Rotation angle of link 2 (rad)

    Returns
    -------
    J : numpy.ndarray
        Jacobian matrix of a 2-link arm in a 2D plane
    """

    J = np.array([[-link1_length*math.sin(theta1) - link2_length*math.sin(theta1+theta2), -link2_length*math.sin(theta1+theta2)],
                  [ link1_length*math.cos(theta1) + link2_length*math.cos(theta1+theta2),  link2_length*math.cos(theta1+theta2)]])
    return J


## <a id='toc5_3_'></a>[Defining the Pseudo-Inverse Matrix](#toc0_)

As mentioned above, when calculating $\Delta \vec{Q}$, the pseudo-inverse of the Jacobian matrix $J$, denoted as $J^+$, is required. Here, we will define a function to compute the pseudo-inverse matrix.

In [6]:
def make_inverse_matrix(mat):
     """
     Calculate the inverse of a 2x2 matrix (used when calculating the pseudo-inverse matrix)
     (Although you can use numpy's linalg.inv function, here we implement it manually to demonstrate the algorithm)

     Parameters
     ----------
     mat: numpy.ndarray
           2x2 matrix

     Returns
     -------
     inverse_mat : numpy.ndarray
                      Inverse of the 2x2 matrix
     """
     inverse_mat = 1/(mat[0,0]*mat[1,1] - mat[0,1]*mat[1,0]) * np.array([[mat[1,1], -mat[0,1]],
                                                                                          [-mat[1,0], mat[0,0]]])
     return inverse_mat

def make_trans_matrix(mat):
     """
     Calculate the transpose of a 2x2 matrix (used when calculating the pseudo-inverse matrix)
     (Although you can use numpy's transpose function, here we implement it manually to demonstrate the algorithm)

     Parameters
     ----------
     mat: numpy.ndarray
           2x2 matrix

     Returns
     -------
     trans_mat : numpy.ndarray
                    Transpose of the 2x2 matrix
     """
     mat_minus =  np.array([[mat[0,0], mat[1,0]],
                                 [mat[0,1], mat[1,1]]])
     return mat_minus

def make_pseudo_inverse_matrix(mat):
     """
     Calculate the pseudo-inverse matrix
     (Although you can use numpy's linalg.pinv function, here we implement it manually to demonstrate the algorithm)

     Parameters
     ----------
     mat: numpy.ndarray
           Matrix

     Returns
     -------
     pseudo_inverse_mat : numpy.ndarray
                               Pseudo-inverse matrix
     """
     trans_mat = make_trans_matrix(mat)
     pseudo_inverse_mat = make_inverse_matrix(trans_mat @ mat) @ trans_mat
     return pseudo_inverse_mat

# <a id='toc6_'></a>[Running the Simulation](#toc0_)

Summarizing the explanation so far, the steps for inverse kinematics using numerical methods with the Jacobian matrix are as follows:
1. Set the "target end-effector position $\vec{P_{\mathrm{goal}}}$".
2. Calculate the "small change in end-effector position $\Delta \vec{P}$" from the "current end-effector position $\vec{P_{\mathrm{current}}}$" towards the "target end-effector position $\vec{P_{\mathrm{goal}}}$".
3. Compute the "Jacobian matrix $J$".
4. Use the pseudo-inverse of the "Jacobian matrix $J$", denoted as $J^+$, to find the "small change in joint angles $\Delta \vec{Q}$".
5. Move the "current joint angles $\vec{Q_{\mathrm{current}}}$" by the "small change in joint angles $\Delta \vec{Q}$" to get the "updated joint angles $\vec{Q_{\mathrm{new}}}$".
6. Based on the "updated joint angles $\vec{Q_{\mathrm{new}}}$", use forward kinematics to find the "updated end-effector position $\vec{P_{\mathrm{new}}}$".
7. Update the "current end-effector position $\vec{P_{\mathrm{current}}}$" to the "updated end-effector position $\vec{P_{\mathrm{new}}}$" and the "current joint angles $\vec{Q_{\mathrm{current}}}$" to the "updated joint angles $\vec{Q_{\mathrm{new}}}$".
8. Repeat steps 2-7 until the "current end-effector position $\vec{P_{\mathrm{current}}}$" converges to the "target end-effector position $\vec{P_{\mathrm{goal}}}$".

By running the following code, inverse kinematics will be executed according to the above steps.


---

Note: For clarity, the movement of the robot arm is visualized in pybullet. However, by commenting out lines 92-94 in the code below, you can omit the visualization part and perform only the inverse kinematics calculations.

Relevant Code Section:
```python
pybullet.resetJointState(arm_id, LINK1_JOINT_IDX, Q_current[0][0]) # Update joint angle
pybullet.resetJointState(arm_id, LINK2_JOINT_IDX, Q_current[1][0]) # Update joint angle
time.sleep(time_step)
```

---


In [7]:
import time
import sys

# Add words to spell checker dictionary
# spell-checker: ignore urdf pybullet

# Length of the robot's links
LINK1_LENGTH = 0.5   # Length of link1 (z-direction length of link1 in "simple2d_arm.urdf")
LINK2_LENGTH = 0.55  # Length of link2 (z-direction length of link2+force_sensor_link in "simple2d_arm.urdf")

# Indices of the robot's joints
LINK1_JOINT_IDX = 0
LINK2_JOINT_IDX = 1

# Initial settings (changing this part will change the results) ========================
# Maximum number of iterations
max_loop_num = 1000

# Initial angles of each link (be careful not to set to a singular posture)
theta1_deg = 0.0
theta2_deg = -1.0

# [Step 1] Target end-effector position (be careful not to specify an unreachable position)
xe_goal = 0.0
ye_goal = 0.8
P_goal = np.array([[xe_goal],
                    [ye_goal]]) # (Initial) target end-effector position P_goal (do not change this part)

# Constant that determines how much the end-effector position P moves in each loop (the larger the value, the faster it converges but becomes unstable; the smaller the value, the slower it converges but becomes stable)
P_delta_param = 0.005

# Threshold to determine if the target end-effector position is reached (how close P_current and P_goal need to be to be considered as reached)
goal_dis = 0.01
# ==========================================================================

# Convert joint angles to radians
link1_angle_rad  = np.deg2rad(theta1_deg) 
link2_angle_rad  = np.deg2rad(theta2_deg)

# Set initial joint angles
Q_current = np.array([[link1_angle_rad],
                      [link2_angle_rad]]) # Current joint angles Q_current

# Set the robot's joint angles to the initial joint angles
pybullet.resetJointState(arm_id, LINK1_JOINT_IDX, Q_current[0][0]) # Update joint angle
pybullet.resetJointState(arm_id, LINK2_JOINT_IDX, Q_current[1][0]) # Update joint angle

# Calculate the initial end-effector position P_current using forward kinematics with homogeneous transformation matrices
H12 = Hz(Q_current[0][0]) @ Hp(LINK1_LENGTH, 0) # H12: link "1" coordinate system -> link "2" coordinate system
H2e = Hz(Q_current[1][0]) @ Hp(LINK2_LENGTH, 0) # H2e: link "2" coordinate system -> "e"nd effector coordinate system
H1e = H12@H2e # H1e: link "1" coordinate system -> "e"nd effector coordinate system
x1, y1 = 0, 0 # Origin of link1 coordinate system
oe = H1e@np.array([x1, y1, 1])
xe, ye = oe[0], oe[1]
P_current = np.array([[xe],
                      [ye]]) # Current end-effector position P_current

# Perform inverse kinematics using numerical methods with the Jacobian matrix
for i in range(max_loop_num):
    # Vector from current end-effector position P_current to target end-effector position P_goal
    P_current_to_P_goal = P_goal - P_current

    # [Step 2]: Small amount ΔP in the direction from current end-effector position P_current to target end-effector position P_goal
    P_delta = P_current_to_P_goal * P_delta_param

    # [Step 3]: Calculate the Jacobian matrix
    jacobian = make_jacobian_matrix(LINK1_LENGTH, Q_current[0][0], LINK2_LENGTH, Q_current[1][0]) # Jacobian matrix J
    
    # Calculate the pseudo-inverse of the Jacobian matrix
    jacobian_inverse = make_pseudo_inverse_matrix(jacobian) 

    # [Step 4]: Calculate the small angles ΔQ required for each joint to move by ΔP
    Q_delta = jacobian_inverse @ P_delta

    # [Step 5]: Move each joint by ΔQ
    Q_new = Q_current + Q_delta
    Q_new = Q_new % (2*math.pi) # Ensure joint angles are within the range of 0 to 2π

    # [Step 6]: Calculate the end-effector position P_new using the updated joint angles Q_new (forward kinematics)
    H12 = Hz(Q_new[0][0]) @ Hp(LINK1_LENGTH, 0) # T12: link "1" coordinate system -> link "2" coordinate system
    H2e = Hz(Q_new[1][0]) @ Hp(LINK2_LENGTH, 0) # T2e: link "2" coordinate system -> "e"nd effector coordinate system
    H1e = H12@H2e # T1e: link "1" coordinate system -> "e"nd effector coordinate system
    x1, y1 = 0, 0 # Origin of link1 coordinate system
    oe = H1e@np.array([x1, y1, 1])
    xe, ye = oe[0], oe[1]
    P_new = np.array([[xe],
                      [ye]]) # Updated end-effector position P_new
    sys.stdout.write("\rTarget end-effector position P_goal: ({:.2f}, {:.2f}), Current end-effector position P_current: ({:.2f}, {:.2f})       ".format(P_goal[0][0], P_goal[1][0], P_current[0][0], P_current[1][0]))

    # [Step 7]: Update current joint angles Q_current and current end-effector position P_current
    Q_current = Q_new
    P_current = P_new

    # Set the robot's joint angles to the current joint angles Q_current
    pybullet.resetJointState(arm_id, LINK1_JOINT_IDX, Q_current[0][0]) # Update joint angle
    pybullet.resetJointState(arm_id, LINK2_JOINT_IDX, Q_current[1][0]) # Update joint angle
    time.sleep(time_step)

    # [Step 8]: Terminate if the target end-effector position P_goal is reached
    if (P_goal[0][0]-P_current[0][0])**2  + (P_goal[1][0]-P_current[1][0])**2 <  goal_dis**2: # Use the formula for the distance between two points (compare squared values to avoid the heavy sqrt function)
        break

# If the number of iterations equals the maximum number of loops, terminate as no solution was found
if i == max_loop_num-1:
    print("\rCould not calculate the position (possible singularity or unreachable coordinates)                ")
else:
    print("\r[Final Result]                                                                                   ")
    print("Number of loops: {}th, Target end-effector position P_goal: ({:.2f}, {:.2f}), Current end-effector position P_current: ({:.2f}, {:.2f})".format(i+1, P_goal[0][0], P_goal[1][0], P_current[0][0], P_current[1][0]))

[Final Result]                                                                                   , 0.79)        
Number of loops: 955th, Target end-effector position P_goal: (0.00, 0.80), Current end-effector position P_current: (0.01, 0.79)
